In [ ]:
import pandas as pd
import numpy as np
import os
from pandas import DataFrame
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import h5py
import gc
import seaborn as sns
import warnings
from scipy import signal
from sklearn.preprocessing import label_binarize
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split,GridSearchCV,LeaveOneOut,cross_val_score,StratifiedKFold
from sklearn import svm,datasets
from sklearn.svm import SVC
from itertools import cycle
from sklearn.metrics import f1_score,classification_report,confusion_matrix, precision_score,recall_score,roc_auc_score, roc_curve, auc, precision_recall_curve
import scipy
from scipy.stats import pearsonr,spearmanr
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from skopt import BayesSearchCV
import joblib
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore")
%matplotlib inline

from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from skopt import BayesSearchCV
from sklearn.model_selection import GridSearchCV
from skopt.callbacks import VerboseCallback, DeadlineStopper, DeltaXStopper

def tiaocanlgb(freq, sh):
    x_true = np.array(freq)
    y_true = np.array(sh).astype(int)
    x_trainy, y_trainy = x_true, y_true
    le = LabelEncoder()
    y_trainy = le.fit_transform(y_trainy)


    bayes_params = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'num_leaves': (2, 50),      
        'max_depth': (0, 50),
        'min_child_samples': (0, 50),
        'max_bin': (100, 1000),
        'subsample': (0.01, 1.0, 'uniform'),
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'min_child_weight': (0, 10),
        'reg_lambda': (1e-9, 1000, 'log-uniform'),
        'reg_alpha': (1e-9, 1.0, 'log-uniform'),
        'scale_pos_weight': (1e-6, 20, 'log-uniform'),
        'n_estimators': (50, 1500),
    }

    model = LGBMClassifier()
    kflod = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
    bayes_opt = BayesSearchCV(estimator=model, search_spaces=bayes_params, cv=kflod, n_jobs=-1, n_iter=32)
    bayes_opt.fit(x_trainy, y_trainy,callback=[VerboseCallback(100), DeadlineStopper(60*5), DeltaXStopper(0.001)])
    best_params = bayes_opt.best_params_
    print('Bayes：{0}'.format(best_params))

    return bayes_opt.best_params_

f_h5z="E:/fenqi/h5zz"
f_sv="E:/fenqi/"
mean_fpr = np.linspace(0, 1, 100)
info=os.listdir(f_h5z)
df=pd.read_csv('E:/fenqi/bestfea/top20output.csv')
features =df['Feature'].unique().tolist()
print(features)
tab=[]
eeg1f,eeg2f,emgf=[],[],[]
for i in range(np.shape(info)[0]):

    domain = os.path.abspath(f_h5z)
    infor = os.path.join(domain,info[i])
    print(infor)
    emg,eeg1,eeg2,scores=readfilez(infor)
    eeg1f.extend(eeg1)
    eeg2f.extend(eeg2)
    emgf.extend(emg)
    tab.extend(scores)
eegall=np.concatenate((eeg1f, eeg2f, emgf), axis=1)
eegx=list(i for i in np.arange(1,31))
eegx = [str(x)+ ' Hz'for x in eegx]
f_head2=[]
f_head2.extend(list('Cerebellar EEG '+j for j in eegx))
f_head2.extend(list('Cerebral EEG '+j for j in eegx))
f_head2.extend(list('EMG '+j for j in eegx))  

intersection = np.intersect1d(f_head2, features)

indices = [f_head2.index(i) for i in intersection]
ez=eegall[:,indices]
print(np.shape(ez))
bcanshu=tiaocanlgb(ez,tab)




In [ ]:
import sys
import pandas as pd
import numpy as np
import os
from pandas import DataFrame
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import h5py
import gc
import seaborn as sns
import warnings
from scipy import signal
from sklearn.preprocessing import label_binarize
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split,GridSearchCV,LeaveOneOut,cross_val_score,StratifiedKFold
from sklearn import svm,datasets
from sklearn.svm import SVC
from itertools import cycle
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import f1_score,classification_report,confusion_matrix, precision_score,recall_score,roc_auc_score, roc_curve, auc, precision_recall_curve
import scipy
from scipy.stats import pearsonr,spearmanr
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from skopt import BayesSearchCV
import joblib
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore")
%matplotlib inline
def sen(Y_test,Y_pred,n):#n为分类数
    
    sen = []
    con_mat = confusion_matrix(Y_test,Y_pred)
    for i in range(n):
        tp = con_mat[i][i]
        fn = np.sum(con_mat[i,:]) - tp
        sen1 = tp / (tp + fn)
        sen.append(sen1)
        
    return sen
def spe(Y_test,Y_pred,n):
    
    spe = []
    con_mat = confusion_matrix(Y_test,Y_pred)
    for i in range(n):
        number = np.sum(con_mat[:,:])
        tp = con_mat[i][i]
        fn = np.sum(con_mat[i,:]) - tp
        fp = np.sum(con_mat[:,i]) - tp
        tn = number - tp - fn - fp
        spe1 = tn / (tn + fp)
        spe.append(spe1)
    
    return spe
def tcplot(eeg2f,eeg2fyc,tab,tabyc,can,name,num):
    mean_fpr = np.linspace(0, 1, 100)
    can=dict(can)

    ez=eeg2f
    ezyc=eeg2fyc
    le = LabelEncoder()
    tab = le.fit_transform(tab)
    tabyc = le.fit_transform(tabyc)
    x_train,y_train=np.array(ez),np.array(tab)
    x_test,y_test=np.array(ezyc),np.array(tabyc)
    model = LGBMClassifier(learning_rate=can['learning_rate'],num_leaves=can['num_leaves'],
                           max_depth=can['max_depth'],min_child_samples=can['min_child_samples'],
                           max_bin=can['max_bin'],subsample=can['subsample'],
                           colsample_bytree=can['colsample_bytree'],min_child_weight=can['min_child_weight'],
                           reg_lambda=can['reg_lambda'],reg_alpha=can['reg_alpha'],
                           scale_pos_weight=can['scale_pos_weight'],n_estimators=can['n_estimators'],verbose=-1)

    model.fit(x_train, y_train)

    pred=model.predict(x_test)
    preds=model.predict_proba(x_test)
    score = model.score(x_test, y_test)

    target_names = ['Wakefulness', 'NREM', 'REM']

    y_test1 = label_binarize(y_test, classes=[0, 1, 2])

    fpr={}
    tpr={}
    roc_auc={}
    rocz=[]
    f_head2=[]
    interp_tprf=[]

    pres=[]
    recal=[]
    sens=[]
    spec=[]

    pres.append(precision_score(y_test, pred,average=None))
    recal.append(recall_score(y_test, pred,average=None))
    sens.append(sen(y_test, pred,3))
    spec.append(spe(y_test, pred,3))


    for i in range(3):
    #         print(i)
        fpr[i], tpr[i], _ = roc_curve(y_test1[:, i], preds[:, i])
        rocz.append(auc(fpr[i], tpr[i]))
        roc_auc[i] = auc(fpr[i], tpr[i])
#         print("roc_auc:",roc_auc)
        interp_tprff=np.interp(mean_fpr, fpr[i], tpr[i])
        interp_tprf.append(interp_tprff)
    df_rocz = pd.DataFrame(rocz, columns=['AUC'])
    df_sleep_stages = pd.DataFrame(target_names, columns=['Sleep Stage'])
    dfauc = pd.concat([df_sleep_stages, df_rocz], axis=1)    
    pres=np.array(pres)
    recal=np.array(recal)
    sens=np.array(sens)
    spec=np.array(spec)
    df=pd.DataFrame()
    df1=pd.DataFrame()
    df2=pd.DataFrame()
    df['Data']=pres[:,0]
    df['Sleep_Stage']=['Wakefulness']
    df1['Data']=pres[:,1]
    df1['Sleep_Stage']=['Nrem']
    df2['Data']=pres[:,2]
    df2['Sleep_Stage']=['Rem']
    dfj=pd.DataFrame()
    dfj=dfj._append(df)
    dfj=dfj._append(df1)
    dfj=dfj._append(df2)
    dfj['Cata']='Precision'
    dfj['Sample_number']=num
    dfj['Area']=name
    
    df=pd.DataFrame()
    df1=pd.DataFrame()
    df2=pd.DataFrame()
    df['Data']=recal[:,0]
    df['Sleep_Stage']=['Wakefulness']
    df1['Data']=recal[:,1]
    df1['Sleep_Stage']=['Nrem']
    df2['Data']=recal[:,2]
    df2['Sleep_Stage']=['Rem']
    dfr=pd.DataFrame()
    dfr=dfr._append(df)
    dfr=dfr._append(df1)
    dfr=dfr._append(df2)
    dfr['Cata']='Recall'
    dfr['Sample_number']=num
    dfr['Area']=name    
    
    df=pd.DataFrame()
    df1=pd.DataFrame()
    df2=pd.DataFrame()
    df['Data']=sens[:,0]
    df['Sleep_Stage']=['Wakefulness']
    df1['Data']=sens[:,1]
    df1['Sleep_Stage']=['Nrem']
    df2['Data']=sens[:,2]
    df2['Sleep_Stage']=['Rem']
    dfm=pd.DataFrame()
    dfm=dfm._append(df)
    dfm=dfm._append(df1)
    dfm=dfm._append(df2)
    dfm['Cata']='Sensitivity'
    dfm['Sample_number']=num
    dfm['Area']=name    

    df=pd.DataFrame()
    df1=pd.DataFrame()
    df2=pd.DataFrame()
    df['Data']=spec[:,0]
    df['Sleep_Stage']=['Wakefulness']
    df1['Data']=spec[:,1]
    df1['Sleep_Stage']=['Nrem']
    df2['Data']=spec[:,2]
    df2['Sleep_Stage']=['Rem']
    dft=pd.DataFrame()
    dft=dft._append(df)
    dft=dft._append(df1)
    dft=dft._append(df2)
    dft['Cata']='Specificity'
    dft['Sample_number']=num
    dft['Area']=name
    
    dfz=pd.DataFrame()
    dfz=dfz._append(dfj)
    dfz=dfz._append(dfr)
    dfz=dfz._append(dfm)
    dfz=dfz._append(dft)
    return interp_tprf,score,dfz,dfauc

def xunlian(num):
    interp_tprd=[]
    interp_tprx=[]
    interp_tprj=[]
    dfd,dfd1=[],[]
    dfx,dfx1=[],[]
    dfj,dfj1=[],[]
    aucd,aucq=[],[]
    scored,scoreq=[],[]
    dfc=pd.DataFrame()
    dfaz=pd.DataFrame()
    dfq=pd.DataFrame()
    dfaq=pd.DataFrame()    
    for j in range(30):
        info=os.listdir(f_h5z)
        print(info)
        infoxl = np.random.choice(info,num, replace=False)
        infoyc =np.setdiff1d(info, infoxl)
        tab=[]
        eeg1f,eeg2f,emgf=[],[],[]
        for i in range(np.shape(infoxl)[0]):

            domain = os.path.abspath(f_h5z)
            infor = os.path.join(domain,infoxl[i])

            emg,eeg1,eeg2,scores=readfilez(infor)
            eeg1f.extend(eeg1)
            eeg2f.extend(eeg2)
            emgf.extend(emg)
            tab.extend(scores)

        tabyc=[]
        eeg1fyc,eeg2fyc,emgfyc=[],[],[]
        
        for i in range(np.shape(infoyc)[0]):

            domain = os.path.abspath(f_h5z)
            infor = os.path.join(domain,info[i])
            emg,eeg1,eeg2,scores=readfilez(infor)
            eeg1fyc.extend(eeg1)
            eeg2fyc.extend(eeg2)
            emgfyc.extend(emg)
            tabyc.extend(scores)


        eegall=np.concatenate((eeg1f, eeg2f, emgf), axis=1)     
        eegallyc=np.concatenate((eeg1fyc, eeg2fyc, emgfyc), axis=1)   

        ez=eegall[:,indices]
        ezyc=eegallyc[:,indices]
        print(np.shape(ez))

        interp_tprl,score,df,dfauc=tcplot(ez,ezyc,tab,tabyc,bcanshu,'top20',num)
        interp_tprd.append(interp_tprl)
        scored.append(score)
        dfc=dfc._append(df)
        dfaz= dfaz._append(dfauc)
    print("ni",df)
    fill_plot(interp_tprd,"{}top20".format(num),scored)
    dfc.to_csv(f_sv+"{}top20_report.csv".format(num))
    dfaz['Sample_number']="{}".format(num)   
     
    return dfaz
    
mean_fpr = np.linspace(0, 1, 100)
eegx=list(i for i in np.arange(1,31))
eegx = [str(x)+ ' Hz'for x in eegx]
f_head2=[]
f_head2.extend(list('Cerebellar EEG '+j for j in eegx))
f_head2.extend(list('Cerebral EEG '+j for j in eegx))
f_head2.extend(list('EMG '+j for j in eegx))

intersection = np.intersect1d(f_head2, features)
indices = [f_head2.index(i) for i in intersection]
f_h5z="E:/fenqi/h5zz"
f_sv="E:/fenqi/top20/"
dfa=pd.DataFrame()
for i in [5,10,15]:
    df_auc=xunlian(i)
    dfa=dfa._append(df_auc) 
dfa.to_csv(f_sv+"top20_AUC.csv")  

In [ ]:

import pandas as pd
import numpy as np
import os
from pandas import DataFrame
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import h5py
import gc
import seaborn as sns
import warnings
from scipy import signal
from sklearn.preprocessing import label_binarize
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split,GridSearchCV,LeaveOneOut,cross_val_score,StratifiedKFold
from sklearn import svm,datasets
from sklearn.svm import SVC
from itertools import cycle
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import f1_score,classification_report,confusion_matrix, precision_score,recall_score,roc_auc_score, roc_curve, auc, precision_recall_curve
import scipy
from scipy.stats import pearsonr,spearmanr
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from skopt import BayesSearchCV
import joblib
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore")
%matplotlib inline

fcsv5="E:/fenqi/top20/5top20_report.csv"
fcsv10="E:/fenqi/top20/10top20_report.csv"
fcsv15="E:/fenqi/top20/15top20_report.csv"
fsv="E:/fenqi/top20/"
df=pd.DataFrame()
df5=pd.read_csv(fcsv5, usecols=['Data', 'Sleep_Stage', 'Cata', 'Sample_number'])
df10=pd.read_csv(fcsv10, usecols=['Data', 'Sleep_Stage', 'Cata', 'Sample_number'])
df15=pd.read_csv(fcsv15, usecols=['Data', 'Sleep_Stage', 'Cata', 'Sample_number'])
df=df._append(df5)
df=df._append(df10)
df=df._append(df15)

sns.set_theme(style="whitegrid",font='Times New Roman',font_scale=2,rc={'font.weight': 'bold'})
dfzzz=df
colors=['#E6873E','#53A362','#DA3F34']
#presicion
g = sns.catplot(
     data=dfzzz[dfzzz['Cata']=="Precision"], x="Sleep_Stage", y='Data', hue='Sample_number',
    palette=colors, errorbar="se",scale='width',
    kind="violin"
)

for ax in g.axes.flat:
    ax.set_xticklabels([])
    ax.set_xlabel('', fontdict={'family': 'Times New Roman', 'size': 26,'weight':'bold'})
    ax.set_ylabel('Precision rate', fontdict={'family': 'Times New Roman', 'size': 26,'weight':'bold'})
    ax.set_yticks(np.arange(0.4,1.2,0.2))
    ax.set_ylim(0.4,1.19)

g._legend.set_title("")
g.legend.set_visible(False)

g.despine(left=True)

plt.savefig(fsv+'Precision.jpg',dpi=300,bbox_inches = 'tight')


#Recall
g = sns.catplot(
    data=dfzzz[dfzzz['Cata']=="Recall"], x="Sleep_Stage", y='Data', hue='Sample_number',
    palette=colors, errorbar="se",scale='width',
    kind="violin"
)

for ax in g.axes.flat:
    ax.set_xticklabels([])
    ax.set_xlabel('', fontdict={'family': 'Times New Roman', 'size': 26,'weight':'bold'})
    ax.set_ylabel('Recall/sensitivity rate', fontdict={'family': 'Times New Roman', 'size': 26,'weight':'bold'})
    ax.set_yticks(np.arange(0.4,1.2,0.2))
    ax.set_ylim(0.4,1.19)

g._legend.set_title("")
g.set_titles(col_template='')
g.legend.set_visible(False)
g.despine(left=True)

plt.savefig(fsv+'Recall.jpg',dpi=300,bbox_inches = 'tight')

#Specificity
g = sns.catplot(
    data=dfzzz[dfzzz['Cata']=="Specificity"], x='Sleep_Stage', y='Data', hue="Sample_number",
    palette=colors, errorbar="se",scale='width',
    kind="violin"
)

for ax in g.axes.flat:
    ax.set_xticklabels([])
    ax.set_xlabel('', fontdict={'family': 'Times New Roman', 'size': 26,'weight':'bold'})
    ax.set_ylabel('Specificity rate', fontdict={'family': 'Times New Roman', 'size': 26,'weight':'bold'})
    ax.set_yticks(np.arange(0.4,1.2,0.2))
    ax.set_ylim(0.4,1.19)

g._legend.set_title("")
g.set_titles(col_template='')
g.legend.set_visible(False)
g.despine(left=True)
plt.savefig(fsv+'Specificity.jpg',dpi=300,bbox_inches = 'tight')

from scipy import stats 
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
pd.set_option('display.float_format',lambda x : '%.3f' % x)
np.set_printoptions(suppress=True)
def print_pvalues_and_stars(df):
    res = pairwise_tukeyhsd(df['value'], df['Sample_number'])
    comparisons = res._results_table.data[1:]  

    for comparison in comparisons:
        print(comparison)
        group1, group2, pvalue = comparison[0], comparison[1], comparison[3]
        stars = ''
        if pvalue < 0.001: stars = '***'
        elif pvalue < 0.01: stars = '**'
        elif pvalue < 0.05: stars = '*'
        print(f"{group1} vs {group2}: pvalue = {pvalue:.4f} {stars}")
        
for i in ['Wakefulness','Nrem','Rem']:
    print(i)
    dftest=dfzzz[(dfzzz['Cata']=="Precision")& (dfzzz['Sleep_Stage']==i)].melt('Sample_number','Data')
    modeltest=ols('value~C(Sample_number)',dftest).fit()
    anova_table=anova_lm(modeltest)

    print(anova_table['F'][0])    
    print(np.float16(anova_table['PR(>F)'][0]))    

    print_pvalues_and_stars(dftest)
for i in ['Wakefulness','Nrem','Rem']:
    print(i)
    dftest=dfzzz[(dfzzz['Cata']=="Recall")& (dfzzz['Sleep_Stage']==i)].melt('Sample_number','Data')
    modeltest=ols('value~C(Sample_number)',dftest).fit()
    anova_table=anova_lm(modeltest)

    print(anova_table['F'][0])    
    print(np.float16(anova_table['PR(>F)'][0]))    

    print_pvalues_and_stars(dftest)
for i in ['Wakefulness','Nrem','Rem']:
    print(i)
    dftest=dfzzz[(dfzzz['Cata']=="Specificity")& (dfzzz['Sleep_Stage']==i)].melt('Sample_number','Data')
    modeltest=ols('value~C(Sample_number)',dftest).fit()
    anova_table=anova_lm(modeltest)

    print(anova_table['F'][0])    
    print(np.float16(anova_table['PR(>F)'][0]))    

    print_pvalues_and_stars(dftest)

In [ ]:
fsv="E:/fenqi/top20/"
dfz=pd.DataFrame()
fcsv="E:/fenqi/top20/top20_AUC.csv"

df=pd.read_csv(fcsv)
dfz= df.drop(df.columns[0], axis=1)  
sns.set_theme(style="whitegrid",font='Times New Roman',font_scale=1.0,rc={'font.weight': 'bold'})
colors=['#E6873E','#53A362','#DA3F34']
g = sns.catplot(
    data=dfz, x='Sleep Stage', y='AUC', hue='Sample_number',
    palette=colors, errorbar="se",scale='width',
    kind="violin"
)

for ax in g.axes.flat:
    ax.set_xticklabels([])
    ax.set_xlabel('', fontdict={'family': 'Times New Roman', 'size': 28,'weight':'bold'})
    ax.set_ylabel('Mean AUC (%)', fontdict={'family': 'Times New Roman', 'size': 28,'weight':'bold'})
    ax.set_yticks(np.arange(0.85,1.04,0.05))
    ax.set_ylim(0.85,1.05)
    ax.tick_params(labelsize=24)

g._legend.set_title("")
g.set_titles(col_template='')
g.legend.set_visible(False)
g.despine(left=True)
plt.savefig(fsv+'AUC.jpg',dpi=300,bbox_inches = 'tight')
from scipy import stats 
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
pd.set_option('display.float_format',lambda x : '%.3f' % x)
np.set_printoptions(suppress=True)
def print_pvalues_and_stars(df):
    res = pairwise_tukeyhsd(df['value'], df['Sample_number'])
    comparisons = res._results_table.data[1:]

    for comparison in comparisons:
        print(comparison)
        group1, group2, pvalue = comparison[0], comparison[1], comparison[3]
        stars = ''
        if pvalue < 0.001: stars = '***'
        elif pvalue < 0.01: stars = '**'
        elif pvalue < 0.05: stars = '*'
        print(f"{group1} vs {group2}: pvalue = {pvalue:.4f} {stars}")
        
for i in ['Wakefulness','NREM','REM']:
    print(i)
    dftest=dfz[(dfz['Sleep Stage']==i)].melt('Sample_number','AUC')
    modeltest=ols('value~C(Sample_number)',dftest).fit()
    anova_table=anova_lm(modeltest)

    print(anova_table['F'][0])    
    print(np.float16(anova_table['PR(>F)'][0]))    

    print_pvalues_and_stars(dftest)